In [5]:
# Created or modified on May 2022
# Author: 임일
# 협업필터링(CF) 추천 - 기본

import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('C:/RecoSys/Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
          'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
          'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
          'Thriller', 'War', 'Western']
movies = pd.read_csv('C:/RecoSys/Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('C:/RecoSys/Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)
# movie ID와 title 빼고 다른 데이터 제거
movies = movies[['movie_id', 'title']]

# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)

# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


In [6]:

##### (1)

# train set의 모든 가능한 사용자 pair의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
user_similarity

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.079049,0.049737,0.050883,0.244962,0.280769,0.362430,0.233847,0.062589,0.280764,...,0.245720,0.074959,0.153007,0.182212,0.155686,0.116487,0.218190,0.107583,0.146127,0.328366
2,0.079049,1.000000,0.097390,0.173297,0.064020,0.196234,0.057951,0.091683,0.074975,0.114488,...,0.141021,0.263681,0.270196,0.396682,0.242568,0.232568,0.176268,0.176754,0.141989,0.078155
3,0.049737,0.097390,1.000000,0.373121,0.000000,0.028862,0.032712,0.094499,0.079732,0.065055,...,0.014023,0.019488,0.145996,0.074081,0.094997,0.000000,0.168859,0.048946,0.065219,0.020115
4,0.050883,0.173297,0.373121,1.000000,0.000000,0.056866,0.046254,0.136795,0.059406,0.038171,...,0.024379,0.050820,0.116534,0.179386,0.085185,0.000000,0.179340,0.140404,0.071194,0.000000
5,0.244962,0.064020,0.000000,0.000000,1.000000,0.216782,0.305629,0.191845,0.045385,0.146924,...,0.224141,0.073634,0.074656,0.052347,0.102588,0.054017,0.167890,0.116008,0.103363,0.210043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.116487,0.232568,0.000000,0.000000,0.054017,0.087579,0.072972,0.099644,0.000000,0.060772,...,0.043127,0.338029,0.187411,0.197237,0.365355,1.000000,0.045322,0.124189,0.039183,0.061244
940,0.218190,0.176268,0.168859,0.179340,0.167890,0.281556,0.244894,0.140599,0.068641,0.243233,...,0.250772,0.081635,0.116439,0.085554,0.074470,0.045322,1.000000,0.064749,0.197072,0.147088
941,0.107583,0.176754,0.048946,0.140404,0.116008,0.103952,0.027255,0.076826,0.000000,0.072648,...,0.035598,0.111311,0.269227,0.229390,0.244087,0.124189,0.064749,1.000000,0.135056,0.051554


In [ ]:

# 주어진 영화의 (movie_id) 가중평균 rating을 계산하는 함수, 
# 가중치는 주어진 사용자와 다른 사용자 간의 유사도(user_similarity)
def CF_simple(user_id, movie_id):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.dropna()
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
        # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating

# 정확도 계산
score(CF_simple)

